# Training Model

#### Importing liabraries

In [28]:
import pickle
import music21
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

#### Importing notes

In [12]:
with open('data/notes.pickle', 'rb') as f:
    notes = pickle.load(f)

pitchNames = set (notes)
nNotes = len (pitchNames)
pitchNames = sorted (pitchNames)

#### Encoding notes

In [13]:
noteCoding = {}

code = 0
for i in pitchNames :
    noteCoding [i] = code
    code += 1
noteCoding

{'0': 0,
 '0.1': 1,
 '0.1.5': 2,
 '0.1.6': 3,
 '0.2': 4,
 '0.2.3.7': 5,
 '0.2.4.7': 6,
 '0.2.5': 7,
 '0.2.6': 8,
 '0.2.7': 9,
 '0.3': 10,
 '0.3.5': 11,
 '0.3.5.8': 12,
 '0.3.6': 13,
 '0.3.6.8': 14,
 '0.3.6.9': 15,
 '0.3.7': 16,
 '0.4': 17,
 '0.4.5': 18,
 '0.4.6': 19,
 '0.4.7': 20,
 '0.5': 21,
 '0.5.6': 22,
 '0.6': 23,
 '1': 24,
 '1.2': 25,
 '1.2.4.6.8.10': 26,
 '1.2.6': 27,
 '1.2.6.8': 28,
 '1.3': 29,
 '1.3.5': 30,
 '1.3.5.8': 31,
 '1.3.6': 32,
 '1.3.7': 33,
 '1.3.8': 34,
 '1.4': 35,
 '1.4.6': 36,
 '1.4.6.9': 37,
 '1.4.7': 38,
 '1.4.7.10': 39,
 '1.4.7.9': 40,
 '1.4.8': 41,
 '1.5': 42,
 '1.5.8': 43,
 '1.5.9': 44,
 '1.6': 45,
 '1.7': 46,
 '10': 47,
 '10.0': 48,
 '10.0.2.5': 49,
 '10.0.3': 50,
 '10.0.4': 51,
 '10.0.5': 52,
 '10.1': 53,
 '10.1.3': 54,
 '10.1.3.5.6': 55,
 '10.1.3.6': 56,
 '10.1.4': 57,
 '10.1.4.6': 58,
 '10.1.5': 59,
 '10.11': 60,
 '10.11.3': 61,
 '10.11.3.5': 62,
 '10.2': 63,
 '10.2.3': 64,
 '10.2.4': 65,
 '10.2.5': 66,
 '10.3': 67,
 '11': 68,
 '11.0': 69,
 '11.0.4': 70,
 

#### One hot encoding

In [26]:
sequence = 100
nInput = []
nOutput = []

for i in range (0, len (notes) - sequence, 1) : 
    
    sIn = notes [i : i + sequence]
    sOut = notes [i + sequence]
    nInput.append ([noteCoding [c]for c in sIn])
    nOutput.append (noteCoding [sOut])

In [17]:
nInput = np.reshape (nInput, (len (nInput), sequence, 1))
print (nInput.shape)
nInput = nInput / float (nNotes)
print (nInput.shape)

(57077, 100, 1)
(57077, 100, 1)


In [29]:
nOutput = np_utils.to_categorical(nOutput)
print (nOutput.shape)

(57077, 358)


#### Creating model

In [ ]:
model = Sequential ()

model.add (LSTM (512, 
    input_shape = (nInput.shape [1], nInput.shape [2]), 
    return_sequences = True
))
model.add (Dropout (0.25))
model.add (LSTM (512, return_sequences = True))
model.add (Dense (256))
model.add (Dropout (0.25))
model.add (LSTM (256))
model.add (Dense (nNotes))
model.add (Activation ('softmax'))

model.compile (loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
model.summary ()

#### Saving weights

In [ ]:
filepath = "weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

#### Training model

In [ ]:
model.fit(nInput, nOutput, epochs = 100, batch_size = 100, callbacks = callbacks_list)

#### Model trained